# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,10.24,79,100,6.91,SJ,1712638640
1,1,crane,31.3974,-102.3501,67.68,77,71,18.34,US,1712638640
2,2,waitangi,-43.9535,-176.5597,58.82,82,87,5.01,NZ,1712638640
3,3,tolanaro,-25.0319,46.9987,73.47,94,40,5.75,MG,1712638640
4,4,ponta delgada,37.7333,-25.6667,65.19,100,75,8.05,PT,1712638641


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',
    colorbar=True,
    title='City Humidity',
    hover_cols=['City', 'Country', 'Humidity']
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Define criteria for your ideal weather condition
ideal_criteria = {
    "Max Temp": 80,  # Maximum temperature (Fahrenheit)
    "Humidity": 50,  # Humidity (%)
    "Cloudiness": 0,  # Cloudiness (%)
    "Wind Speed": 10  # Wind speed (mph)
}

# Narrow down cities that fit the criteria
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] <= ideal_criteria["Max Temp"]) &
    (city_data_df["Humidity"] <= ideal_criteria["Humidity"]) &
    (city_data_df["Cloudiness"] <= ideal_criteria["Cloudiness"]) &
    (city_data_df["Wind Speed"] <= ideal_criteria["Wind Speed"])
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
188,188,mingyue,43.1069,128.9217,50.16,12,0,5.37,CN,1712638675
270,270,laguna,38.4210,-121.4238,60.76,45,0,8.05,US,1712638689
281,281,ungsang-nodongjagu,42.3578,130.4622,45.95,25,0,5.75,KP,1712638696
415,415,san gabriel,34.0961,-118.1058,65.48,46,0,1.01,US,1712638569
490,490,huangnihe,43.5583,128.0239,46.20,15,0,6.46,CN,1712638739


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,longyearbyen,SJ,78.2186,15.6401,79,
1,crane,US,31.3974,-102.3501,77,
2,waitangi,NZ,-43.9535,-176.5597,82,
3,tolanaro,MG,-25.0319,46.9987,94,
4,ponta delgada,PT,37.7333,-25.6667,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    "categories": "accommodation",
    "radius": radius,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Obtener la latitud y longitud de la ciudad actual del DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Agregar parámetros de filtro y sesgo con la latitud y longitud de la ciudad actual al diccionario params
    params["filter"] = f"point: {lng},{lat}"
    params["bias"] = f"point: {lng},{lat}"
    
    # URL base
    base_url = "https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey=18b44705502e44b997d65acbc1a44209"

    # Hacer una solicitud API utilizando el diccionario params
    response = requests.get(base_url, params=params)
    
    # Convertir la respuesta de la API a formato JSON
    data = response.json()
    
    # Obtener el nombre del primer hotel de los resultados y almacenarlo en el DataFrame hotel_df
    try:
        hotel_name = data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # Si no se encuentra ningún hotel, establecer el nombre del hotel como "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Registrar los resultados de la búsqueda
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

import folium

# Inicializar el mapa en una ubicación central
mapa = folium.Map(location=[hotel_df["Lat"].mean(), hotel_df["Lng"].mean()], zoom_start=5)

# Iterar a través del DataFrame de hoteles para agregar marcadores al mapa
for index, row in hotel_df.iterrows():
    # Verificar si se encontró un hotel
    if row["Hotel Name"] != "No hotel found":
        # Añadir un marcador en la ubicación del hotel con el nombre del hotel como popup
        folium.Marker([row["Lat"], row["Lng"]], popup=row["Hotel Name"]).add_to(mapa)

# Mostrar el mapa
mapa



Starting hotel search


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
# Configure the map plot with hotel name and country as additional hover information
map_plot = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',
    colorbar=True,
    title='City Humidity with Nearest Hotel',
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name']  # Agrega 'Hotel Name' al mensaje de hover
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)